In [4]:
from sqlalchemy import create_engine, text
import pandas as pd
import os
from dotenv import load_dotenv

# Load env variables
load_dotenv()

# Get DB credentials
db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")
db_host = os.getenv("DB_HOST")
db_port = os.getenv("DB_PORT")
db_name = os.getenv("DB_NAME")

# Set up Postgres connection
engine = create_engine(f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")

# Show all rows in output
pd.set_option('display.max_rows', None)

### Business Question:
What are the most discussed Reddit themes about Snap over the years?

Understanding which concerns (e.g., layoffs, culture, AI) have persisted or spiked across different years helps Snap prioritize internal strategy and reputation management.

In [3]:
query_1 = '''
SELECT
    year,
    topic_tag,
    COUNT(*) AS post_count
FROM reddit_posts
GROUP BY year, topic_tag
ORDER BY year, post_count DESC;
'''

df_q1 = pd.read_sql(query_1, engine)
df_q1

,year,topic_tag,post_count
0,2012,interview,1
1,2013,AI roles,1
2,2014,AI roles,4
3,2014,interview,4
4,2015,AI roles,5
5,2015,interview,5
6,2015,layoff,1
7,2016,interview,19
8,2016,AI roles,13
9,2016,other,4


**Insight:**  
Mentions of interviews related to Snap have increased sharply over time, rising from 82 posts in 2021 to 193 in 2024 — a 135% increase. Similarly, layoff-related posts jumped from 5 in 2021 to 47 in 2024, representing a 9x spike. Meanwhile, AI roles remained consistently high from 2022 to 2024, with 99 mentions in 2024, slightly down from the peak of 109 in 2022.

**Recommendation:**  
Snap’s hiring team should proactively address job candidate uncertainty around layoffs and interviews by improving the transparency of their hiring process and providing clearer job security signals. A dedicated “Hiring FAQs” section or more active recruiter presence on platforms like Reddit could help ease concerns and build trust.

**Prediction:**  
If current sentiment trends continue, public commentary on Snap’s hiring experience and job stability will likely influence offer acceptance rates. As competition for ML/AI talent grows, even a minor perceived instability could push strong candidates toward more transparent competitors.

----------------------------------------------------------------------------------------------------

### Business Question:

Which subreddits are driving the most discussion about Snap-related layoffs, AI roles, culture, and interviews?

In [ ]:
sql_query = '''
SELECT
    subreddit,
    topic_tag,
    COUNT(*) AS post_count
FROM reddit_posts
GROUP BY subreddit, topic_tag
ORDER BY topic_tag, post_count DESC;
'''
result = pd.read_sql(sql_query, con=engine)
print(result)

            subreddit  topic_tag  post_count
0   cscareerquestions   AI roles         103
1          WorkReform   AI roles          73
2                jobs   AI roles          69
3         internships   AI roles          54
4     experienceddevs   AI roles          53
5            csMajors   AI roles          52
6        overemployed   AI roles          50
7      recruitinghell   AI roles          42
8         techcareers   AI roles           4
9                jobs    culture           9
10         WorkReform    culture           4
11    experienceddevs    culture           4
12  cscareerquestions    culture           3
13       overemployed    culture           2
14           csMajors    culture           2
15  cscareerquestions  interview         183
16     recruitinghell  interview         150
17               jobs  interview         125
18           csMajors  interview         105
19    experienceddevs  interview          93
20        internships  interview          58
21       o

**Insight:**  
Out of all AI-related posts (556 total), r/cscareerquestions holds 18.5%, followed by r/WorkReform (13.1%) and r/jobs (12.4%).  
Interview-tagged posts (855 total) are even more concentrated: r/cscareerquestions alone accounts for 21.4%, while r/recruitinghell contributes 17.5%.  
Layoff mentions are more dispersed, with both r/cscareerquestions and r/WorkReform making up around 15–16% of the 250 total layoff posts.

**Recommendation:**  
Prioritize r/cscareerquestions, r/recruitinghell, and r/WorkReform as key listening hubs. These communities surface direct concerns about Snap's hiring process, layoffs, and AI team perception.

**Prediction:**  
As Snap increases AI hiring, discussion in r/cscareerquestions will likely surge.  
If sentiment skews negative, this may impact how competitive candidates perceive the company, especially for hard-to-fill ML/AI roles.


----------------------------------------------------------------------------------------------------

### Business Question:

Which Snap-related posts on Reddit received the highest engagement (upvotes and comments) across each major theme, and which subreddits did they appear in?

In [7]:
sql_query = '''
WITH topic_summary AS (
    SELECT
        id,
        topic_tag,
        score,
        num_comments,
        created_utc,
        CASE
            WHEN created_utc >= '2023-01-01' THEN 'recent'
            ELSE 'older'
        END AS era
    FROM reddit_posts
    WHERE topic_tag IS NOT NULL
),
exploded AS (
    SELECT 
        id,
        UNNEST(STRING_TO_ARRAY(topic_tag, ',')) AS tag,
        score,
        num_comments,
        era
    FROM topic_summary
),
topic_aggregates AS (
    SELECT
        tag,
        era,
        COUNT(*) AS post_count,
        AVG(score) AS avg_score,
        AVG(num_comments) AS avg_comments
    FROM exploded
    GROUP BY tag, era
),
ranked_topics AS (
    SELECT *,
           RANK() OVER (PARTITION BY era ORDER BY avg_score DESC) AS score_rank
    FROM topic_aggregates
)

SELECT 
    t1.tag,
    t1.era,
    t1.post_count,
    ROUND(t1.avg_score, 1) AS avg_score,
    ROUND(t1.avg_comments, 1) AS avg_comments,
    t2.total_posts_in_era,
    t1.score_rank
FROM ranked_topics t1
JOIN (
    SELECT era, COUNT(*) AS total_posts_in_era
    FROM exploded
    GROUP BY era
) t2 ON t1.era = t2.era
ORDER BY t1.tag, t1.era;
'''

df_q3 = pd.read_sql(sql_query, engine)
pd.set_option('display.max_rows', None)
df_q3

,tag,era,post_count,avg_score,avg_comments,total_posts_in_era,score_rank
0,AI roles,older,223,391.7,90.1,705,3
1,AI roles,recent,277,766.2,135.8,951,2
2,culture,older,5,84.4,46.8,705,5
3,culture,recent,19,339.3,132.8,951,5
4,interview,older,364,451.1,90.8,705,2
5,interview,recent,449,649.2,114.4,951,3
6,layoff,older,45,785.2,138.2,705,1
7,layoff,recent,118,879.6,148.2,951,1
8,other,older,68,151.3,104.4,705,4
9,other,recent,88,429.8,149.8,951,4


**Insight:**  
Reddit posts about Snap tagged with “layoff” nearly tripled from 45 (older era) to 118 posts (recent era). Average post score also increased from 785 to 880, and average comment count rose from 138 to 148. Similarly, “AI roles” saw a rise in volume (223 → 277) and engagement (score: 392 → 766, comments: 90 → 136). This shows growing interest and concern in both Snap’s layoffs and its AI hiring focus.

**Recommendation:**  
Snap should prepare messaging strategies and recruiter FAQs that address the two highest-signal themes: layoffs and AI roles. High comment activity signals people have questions — meet them where they are with proactive answers.

**Prediction:**  
If engagement continues trending upward, sentiment around layoffs and AI may influence how potential hires perceive job stability and tech direction. Snap risks losing top candidates if it doesn't clarify internal strategy through public-facing channels.